In [81]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import connstr  
import pandas as pd

In [83]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
    SELECT FP.ASPX , FP.PK_FormPage,  FP.Text, MCK.* FROM vw_MetricsCompositeKeys MCK 
    INNER JOIN fsma_FormPages FP ON FP.PK_FormPage=MCK.FK_FormPage
    WHERE PK_FORM = '2021-A-EINSTEIN' AND PK_FormPage <> 2072
""" 
df = pd.read_sql(query,con=conn)  
df
alt = ''
for i,r in df.iterrows():
    text = re.sub('Section \d:' , f'Section {i+1}:' , r['Text'])
    alt = alt + f"""
    UPDATE fsma_FormPages SET Text = '{text}', sortpos = {i+1}, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_{i}.aspx'
    WHERE PK_FormPage = {r['PK_FormPage']}
    """ 
conn.close()
print( f'{alt}' )



    UPDATE fsma_FormPages SET Text = 'Section 1: Public IP Space', sortpos = 1, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_0.aspx'
    WHERE PK_FormPage = 2070
    
    UPDATE fsma_FormPages SET Text = 'Section 2: Unannounced IP Space', sortpos = 2, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_1.aspx'
    WHERE PK_FormPage = 2071
    
    UPDATE fsma_FormPages SET Text = 'Section 3: Border Gateway Protocol (BGP) Information', sortpos = 3, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_2.aspx'
    WHERE PK_FormPage = 2073
    
    UPDATE fsma_FormPages SET Text = 'Section 4: Unmonitored Traffic', sortpos = 4, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_3.aspx'
    WHERE PK_FormPage = 2074
    
    UPDATE fsma_FormPages SET Text = 'Section 5: Einstein Test Access Point (TAP) Locations', sortpos = 5, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_4.aspx'
    WHERE PK_FormPage = 2075
    
    UPDATE fsma_FormPages SET Text = 'Section 6: Network Diagram(s)', sortpos = 6, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_5.aspx'
    WHER

In [88]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
    SELECT QPK FROM vw_MetricsCompositeKeys MCK 
    INNER JOIN fsma_FormPages FP ON FP.PK_FormPage=MCK.FK_FormPage
    WHERE PK_FORM = '2021-A-EINSTEIN' AND PK_FormPage <> 2072
""" 
df = pd.read_sql(query,con=conn)  
df
alt = ''
for i,r in df.iterrows(): 
    alt = alt + f"""
    UPDATE fsma_Questions SET identifier_text = '{i+1}' WHERE PK_Question = {r['QPK']}""" 
conn.close()
print( f'{alt}' )


    UPDATE fsma_Questions SET identifier_text = '1' WHERE PK_Question = 20850
    UPDATE fsma_Questions SET identifier_text = '2' WHERE PK_Question = 20861
    UPDATE fsma_Questions SET identifier_text = '3' WHERE PK_Question = 20881
    UPDATE fsma_Questions SET identifier_text = '4' WHERE PK_Question = 20891
    UPDATE fsma_Questions SET identifier_text = '5' WHERE PK_Question = 20900
    UPDATE fsma_Questions SET identifier_text = '6' WHERE PK_Question = 20940
    UPDATE fsma_Questions SET identifier_text = '7' WHERE PK_Question = 20950
    UPDATE fsma_Questions SET identifier_text = '8' WHERE PK_Question = 20960


In [ ]:
secs = [
      {'s':1, 'name':'EinsteinPublicIP'}
    , {'s':2, 'name':'EinsteinUnannounced'}
    , {'s':3, 'name':'EinsteinBGP'}
    , {'s':4, 'name':'EinsteinUnmonitoredTraffic'}
    , {'s':5, 'name':'EinsteinTAPS'} 
    , {'s':7, 'name':'EinsteinNetChannelsDS'}
    , {'s':8, 'name':'EinsteinNAT'}
]

In [63]:

lst = [] 
for d in secs:
    df = pd.read_csv(f"C:\\Users\\Tim\\Documents\\EINSTEIN\\ss\\{d['s']}.csv")
    # D:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Archive\Old DBUpdates\DB_Update_7.27_2020_A_EINSTEIN_SAOP.sql
    df['SECTION'] = d['s']
    df['TABLE_NAME'] = d['name']
    df['COLUMN_NAME'] = df['Column Headings'].apply(lambda s: re.sub('[^A-Za-z0-9_]','',s))
    df = df.rename(columns={'Column Headings': 'Headings'
                            , 'Enumerated List': 'Enumerate'
                            , 'Data Entry Validations': 'Validations'
                            , 'Additional Help Text': 'Help'})
    lst.append(df)
df_csv = pd.concat(lst)
df_csv 

,Headings,Validations,Enumerate,Help,SECTION,TABLE_NAME,COLUMN_NAME
0,Starting IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,First address of the contiguous IP range in IP...,1,EinsteinPublicIP,StartingIP
1,Ending IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,Last address of the contiguous IP range that c...,1,EinsteinPublicIP,EndingIP
2,TIC/MTIPS,Mandatory Entry\n\nValidation\nMust be one of ...,TIC\nMTIPS\nBoth,EINSTEIN monitoring service for the address bl...,1,EinsteinPublicIP,TICMTIPS
3,Name/Purpose,Optional Entry\n\nValidation\nSubstitute with ...,NaN,A short description of the address block that ...,1,EinsteinPublicIP,NamePurpose
4,Internet Service Provider (ISP),Mandatory Entry\n\nValidation\nMust be one of ...,Not Applicable\nOther\nAcquia\nAdobe\nAkamai\n...,Select the name of the ISP that hosts this IP ...,1,EinsteinPublicIP,InternetServiceProviderISP
5,Other ISP,Optional Entry\n\nValidation\n- If the ?ISP? f...,NaN,If you selected ?Other? for ISP then enter the...,1,EinsteinPublicIP,OtherISP
6,Visible to EINSTEIN?,Mandatory Entry\n\nValidation\nMust be one of ...,Yes\nNo\nUnknown,Is this address block presented to EINSTEIN fo...,1,EinsteinPublicIP,VisibletoEINSTEIN
7,External Organization,Optional Entry\n\nValidation\nSubstitute with ...,NaN,If this address range is associated with an ex...,1,EinsteinPublicIP,ExternalOrganization
0,Starting IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,First address of the contiguous IP range in IP...,2,EinsteinUnannounced,StartingIP
1,Ending IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,Last address of the contiguous IP range that c...,2,EinsteinUnannounced,EndingIP


In [ ]:
alt = ''
for i,r in df_csv.iterrows():
    TABLE_NAME = r['TABLE_NAME']
    for tname in [TABLE_NAME, f'{TABLE_NAME}SNAPSHOT']:
        alt = alt + f"""
        --ALTER TABLE {tname} DROP COLUMN {r['COLUMN_NAME']};
        IF NOT EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{tname}' and COLUMN_NAME='{r['COLUMN_NAME']}')
        BEGIN 
            ALTER TABLE {tname} ADD {r['COLUMN_NAME']} NVARCHAR(255) NULL
        END
        GO
        """
print( f'{alt}' )

In [56]:
tbls = '\'' + '\', \''.join( [s['name'] for s in secs] ) + '\''
print(tbls)


'EinsteinPublicIP', 'EinsteinUnannounced', 'EinsteinBGP', 'EinsteinUnmonitoredTraffic', 'EinsteinTAPS', 'EinsteinNetChannelsDS', 'EinsteinNAT'


In [70]:
engine = create_engine(connstr)
conn = engine.connect()
query = f""" \
    SELECT COLUMN_NAME, TABLE_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME IN({tbls})
""" 
df_schema = pd.read_sql(query,con=conn)   
conn.close()
df_schema.head()

,COLUMN_NAME,TABLE_NAME
0,PK_EinsteinBGP,EinsteinBGP
1,PK_OrgSubmission,EinsteinBGP
2,ASN,EinsteinBGP
3,BGPRoutes,EinsteinBGP
4,BGPPeerIP,EinsteinBGP


In [71]:
df_new = df_csv.loc[ ~df_csv.COLUMN_NAME.isin(df_schema.COLUMN_NAME ) ]
df_new.head()

,Headings,Validations,Enumerate,Help,SECTION,TABLE_NAME,COLUMN_NAME
0,Starting IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,First address of the contiguous IP range in IP...,1,EinsteinPublicIP,StartingIP
1,Ending IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,Last address of the contiguous IP range that c...,1,EinsteinPublicIP,EndingIP
2,TIC/MTIPS,Mandatory Entry\n\nValidation\nMust be one of ...,TIC\nMTIPS\nBoth,EINSTEIN monitoring service for the address bl...,1,EinsteinPublicIP,TICMTIPS
4,Internet Service Provider (ISP),Mandatory Entry\n\nValidation\nMust be one of ...,Not Applicable\nOther\nAcquia\nAdobe\nAkamai\n...,Select the name of the ISP that hosts this IP ...,1,EinsteinPublicIP,InternetServiceProviderISP
6,Visible to EINSTEIN?,Mandatory Entry\n\nValidation\nMust be one of ...,Yes\nNo\nUnknown,Is this address block presented to EINSTEIN fo...,1,EinsteinPublicIP,VisibletoEINSTEIN


In [80]:
df_rem = df_schema.loc[ ~df_schema.COLUMN_NAME.isin( df_csv.COLUMN_NAME ) ]
df_rem

,COLUMN_NAME,TABLE_NAME
0,PK_EinsteinBGP,EinsteinBGP
1,PK_OrgSubmission,EinsteinBGP
2,ASN,EinsteinBGP
3,BGPRoutes,EinsteinBGP
5,NonAdvertised,EinsteinBGP
6,Notes,EinsteinBGP
7,UserId,EinsteinBGP
8,isActive,EinsteinBGP
9,PK_EinsteinNAT,EinsteinNAT
10,PK_OrgSubmission,EinsteinNAT
